In [1]:
#!gsutil -m cp -r gs://neurips-adc-bucket/raw/train/100468857 ../data/raw/train
#!gsutil -m cp -r gs://neurips-adc-bucket/raw/train/1005054328 ../data/raw/train
#!gsutil cp gs://neurips-adc-bucket/raw/* ../data/raw/

In [2]:
from google.cloud import storage
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, StandardOptions
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ibeam
import os
import subprocess
import glob
import shutil

In [3]:
storage_client = storage.Client()

In [4]:
blobs = storage_client.list_blobs(
    bucket_or_name="neurips-adc-bucket", match_glob="raw/**/*", delimiter="/"
)

In [5]:
[blob for blob in blobs.prefixes]

[]

In [6]:
def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        print(blob.name)

In [7]:
def copy_to(element, dest: str):
    sufix = "/".join(element.split("/")[-2:])
    output = os.path.join(dest, sufix)
    shutil.copytree(src=element, dst=output)

In [8]:
with beam.Pipeline() as pipeline:
    outputs = (
        pipeline
        | "Create initial values" >> beam.Create(glob.glob("../data/raw/**/*"))
        | "Copy the folder" >> beam.Map(lambda x: copy_to(x, "../data/output"))
    )
    outputs | beam.Map(print)

ERROR:apache_beam.runners.common:FileExistsError: [Errno 17] File exists: '../data/output/train/100468857' [while running '[8]: Copy the folder']
Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1495, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 688, in apache_beam.runners.common.SimpleInvoker.invoke_process
  File "/home/drxc/.pyenv/versions/3.10.13/envs/neurips/lib/python3.10/site-packages/apache_beam/transforms/core.py", line 2065, in <lambda>
    wrapper = lambda x: [fn(x)]
  File "/tmp/ipykernel_46938/1934613105.py", line 5, in <lambda>
    | "Copy the folder" >> beam.Map(lambda x: copy_to(x, "../data/output"))
  File "/tmp/ipykernel_46938/1526363044.py", line 4, in copy_to
    shutil.copytree(src=element, dst=output)
  File "/home/drxc/.pyenv/versions/3.10.13/lib/python3.10/shutil.py", line 559, in copytree
    return _copytree(entries=entries, src=src, dst=dst, symlinks=symlinks,
  File "/home/drxc/.

RuntimeError: FileExistsError: [Errno 17] File exists: '../data/output/train/100468857' [while running '[8]: Copy the folder']

Next setp appliquer la transformation de bout en bout dans le notebook.
quand ce sera bon on va voir comment intéger cela dans un ExampleGen